## Naivni Bayesov klasifikator
za napovedovanje cene rabljenega avtomobila.

Ceno bom poskušal napovedati iz podatka o znamki, vrsti motorja, vrsti menjalnika, močjo motorja, leta izdelave in števila prevoženih kilometrov.

In [1]:
import pandas as pd

avti = pd.read_csv("../podatki/rabljeni_avtomobili.csv")

# zvezne parametre zaokrožimo
avti.cena = avti.cena.round(-3)
avti.kilovati = avti.kilovati.round(-1)
avti.stevilo_kilometrov = avti.stevilo_kilometrov.round(-5)

Avtomobile razdelimo na učno in testno množico.

In [2]:
import numpy as np
maska = np.random.rand(len(avti)) < 0.8
parametri = ["ime_znamke", "menjalnik", "vrsta_motorja", "kilovati", "leto_izdelave", "stevilo_kilometrov"]
učni_avti = avti[maska][parametri+["cena"]].dropna()
testni_avti = avti[~maska][parametri+["cena"]].dropna()

In [3]:
pojavitve_cen = učni_avti.groupby("cena").size()
verjetnosti_cen = pojavitve_cen / len(učni_avti)

$p_i$ ... i-ti parameter  
$c$ ... cena
$$P(P_i = p_i \mid C = c)$$

In [4]:
pogojne_verjetnosti_parametrov = pd.concat(
    [
        (učni_avti.groupby(["cena", parameter]).size() / pojavitve_cen)
        .unstack().transpose().fillna(1/len(učni_avti))
        for parameter
        in parametri
    ],
    keys=parametri
)

In [5]:
def napovej_ceno(avtomobil: pd.Series):
    indeksi = set(zip(avtomobil.index, avtomobil))
    verjetne_cene = pogojne_verjetnosti_parametrov[
        pogojne_verjetnosti_parametrov.index.isin(indeksi)
    ].product(numeric_only=True)*verjetnosti_cen
    verjetne_cene /= verjetne_cene.sum()
    return (verjetne_cene.index * verjetne_cene).sum().round(-2)

In [6]:
testni_avti["napovedana_cena"] = testni_avti.apply(napovej_ceno, axis=1)

Povprečna napaka napovedi

In [7]:
RMSE = (((testni_avti.napovedana_cena-testni_avti.cena)**2).sum()/len(testni_avti))**0.5
RMSE

7319.369418019654

In [8]:
testni_avti.sample(10)

,ime_znamke,menjalnik,vrsta_motorja,kilovati,leto_izdelave,stevilo_kilometrov,cena,napovedana_cena
12031,Nissan,ROČNI,BENCIN,60.0,2008.0,200000.0,3000.0,3800.0
9356,Mazda,AVTOMATSKI,DIEZEL,130.0,2017.0,200000.0,22000.0,19400.0
10543,Mercedes-Benz,AVTOMATSKI,DIEZEL,150.0,2013.0,300000.0,17000.0,12600.0
1213,Audi,ROČNI,DIEZEL,110.0,2018.0,100000.0,19000.0,20500.0
789,Audi,ROČNI,DIEZEL,100.0,2012.0,200000.0,13000.0,10000.0
11842,Mitsubishi,AVTOMATSKI,DIEZEL,110.0,2017.0,100000.0,17000.0,24200.0
7340,Ford,AVTOMATSKI,DIEZEL,110.0,2017.0,200000.0,13000.0,20000.0
15609,Renault,ROČNI,BENCIN,80.0,2009.0,200000.0,3000.0,4500.0
14802,Renault,ROČNI,DIEZEL,80.0,2017.0,100000.0,12000.0,15000.0
19219,Volkswagen,ROČNI,DIEZEL,110.0,2013.0,300000.0,12000.0,11100.0
